# AIB to Bambora SF Cost Comparison Application

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import sys
print (sys.version)

2.7.12 |Anaconda 4.2.0 (64-bit)| (default, Jun 29 2016, 11:07:13) [MSC v.1500 64 bit (AMD64)]


## Import the Necessary Libraries
Below are the libraries required to assemble the project.

In [2]:
import pyodbc #CONNECT VIA PYODBC IMPORT THE PYODBC LIBRARY
import pandas as pd #CONNECT TO THE PANDAS LIBRARY
import numpy as np #IMPORT NUMPY
import csv
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
pd.options.mode.chained_assignment = None  # default='warn'
#print pyodbc.dataSources()
#conn=pyodbc.connect("Driver={NetezzaSQL};SERVER=10.80.10.70;PORT=5480;DATABASE=P_FDW;UID=djohnson;PWD=Sup3rm4n",ansi=True)

## Bambora Import

In [3]:
bb_column_names=['TRANSACTION_REF','ADDITIONAL_REFERENCE_2','PAYMENTREFERENCE','BB_TRANSACTION_TYPE','MCC','CONTRACT_ID','IIN','BIN_DESC','ORDERID','PAYMENT_PROCESSOR','MONTH','DAY','YEAR','PERIOD'
              ,'ISSUER_COUNTRY','MERCHANT_COUNTRY','TRANSACTION_TYPE','CARD_SCHEME','CREDIT_DEBIT','CONSUMER_CORPORATE','SECURITY_LEVEL','REGIONALITY'
              ,'PRODUCT_CODE','SF_FIXED_FEE','SF_PERC_FEE','CON_COM_BIN','VOLUME','ROUNDED_TRXN_AMOUNT','SCHEME_TOTAL_FEE_EUR','FLOW','BB_IC_TOTAL','BB_IC_BPS','BB_SF_BPS']

In [4]:
#import the file from local drives
#bb=pd.read_csv('d:\\djohnson\\Desktop\\aib_comp_pull_bb_data.csv',error_bad_lines=False,skiprows=32,sep=';',names=bb_column_names,index_col=False)
bb=pd.read_csv('d:\\djohnson\\Desktop\\aib_comp_pull_bb_data.csv',error_bad_lines=False,sep=';',skiprows=95,names=bb_column_names,index_col=False)

In [5]:
bb['BB_TRANSACTION_TYPE']=bb['BB_TRANSACTION_TYPE'].str.strip()
bb['ISSUER_COUNTRY']=bb['ISSUER_COUNTRY'].str.strip()
bb['MERCHANT_COUNTRY']=bb['MERCHANT_COUNTRY'].str.strip()
bb['CARD_SCHEME']=bb['CARD_SCHEME'].str.strip()
bb['CREDIT_DEBIT']=bb['CREDIT_DEBIT'].str.strip()
bb['CONSUMER_CORPORATE']=bb['CONSUMER_CORPORATE'].str.strip()
bb['REGIONALITY']=bb['REGIONALITY'].str.strip()
bb['BIN_DESC']=bb['BIN_DESC'].str.strip()

In [7]:
#create a new copy and safe old in memory for working purposes and to prevent repeated import from local drive
bb1=bb.copy()
bb1=bb1.dropna()
bb1=bb1.drop(['CON_COM_BIN'],axis=1)

In [8]:
bb1['SECURITY_LEVEL'].unique()

array([ 7.,  5.,  6.])

In [9]:
bb1['SECURITY_LEVEL']=bb1['SECURITY_LEVEL'].apply(lambda x:'NON-SECURE' if (7 == x) or (0 == x) or (6 == x) else 'SECURE')

In [10]:
bb1=bb1[(bb1['BB_TRANSACTION_TYPE']=='Sale')]
bb1=bb1.rename({'ROUNDED_TRXN_AMOUNT':'FLOW'})
bb1.head()

,TRANSACTION_REF,ADDITIONAL_REFERENCE_2,PAYMENTREFERENCE,BB_TRANSACTION_TYPE,MCC,CONTRACT_ID,IIN,BIN_DESC,ORDERID,PAYMENT_PROCESSOR,...,PRODUCT_CODE,SF_FIXED_FEE,SF_PERC_FEE,VOLUME,ROUNDED_TRXN_AMOUNT,SCHEME_TOTAL_FEE_EUR,FLOW,BB_IC_TOTAL,BB_IC_BPS,BB_SF_BPS
1,226044368911,2.260444e+11,2.260444e+11,Sale,5399.0,5859.0,497886.0,CLASSIC,7.200080e+09,235.0,...,100440.0,0.004761,0.00024,1.0,187.0,0.049561,186.670000,0.560010,0.003000,0.000266
2,226051366113,2.260514e+11,2.260514e+11,Sale,5399.0,5859.0,535218.0,CLASSIC,7.200093e+09,235.0,...,100450.0,0.120247,0.00486,1.0,40.0,0.316400,40.360000,0.660000,0.016353,0.007839
3,226070415771,2.260704e+11,2.260704e+11,Sale,5399.0,5859.0,522661.0,CLASSIC,9.221370e+09,235.0,...,100455.0,0.165574,0.00081,1.0,23.0,0.034386,22.566265,0.067115,0.002974,0.001524
4,226061413145,2.260614e+11,2.260614e+11,Sale,5399.0,5859.0,497040.0,CLASSIC,7.200195e+09,235.0,...,100440.0,0.004761,0.00020,1.0,25.0,0.009729,24.840000,0.049680,0.002000,0.000392
5,225514628687,2.255146e+11,2.255146e+11,Sale,4814.0,9391.0,465858.0,CLASSIC,4.000023e+09,235.0,...,100441.0,0.004391,0.00020,1.0,7.0,0.007236,6.710923,0.013422,0.002000,0.001078


In [11]:
bb1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 358731 entries, 1 to 358731
Data columns (total 32 columns):
TRANSACTION_REF           358731 non-null object
ADDITIONAL_REFERENCE_2    358731 non-null float64
PAYMENTREFERENCE          358731 non-null float64
BB_TRANSACTION_TYPE       358731 non-null object
MCC                       358731 non-null float64
CONTRACT_ID               358731 non-null float64
IIN                       358731 non-null float64
BIN_DESC                  358731 non-null object
ORDERID                   358731 non-null float64
PAYMENT_PROCESSOR         358731 non-null float64
MONTH                     358731 non-null float64
DAY                       358731 non-null float64
YEAR                      358731 non-null float64
PERIOD                    358731 non-null object
ISSUER_COUNTRY            358731 non-null object
MERCHANT_COUNTRY          358731 non-null object
TRANSACTION_TYPE          358731 non-null object
CARD_SCHEME               358731 non-null obje

In [12]:
bb1_contracts=bb1[['CONTRACT_ID','ISSUER_COUNTRY','MERCHANT_COUNTRY','CARD_SCHEME','CREDIT_DEBIT','CONSUMER_CORPORATE','SECURITY_LEVEL','IIN','BIN_DESC']]

In [13]:
bb1=bb1[(bb1['BIN_DESC']!='')]

In [14]:
bb1_contracts.head()

,CONTRACT_ID,ISSUER_COUNTRY,MERCHANT_COUNTRY,CARD_SCHEME,CREDIT_DEBIT,CONSUMER_CORPORATE,SECURITY_LEVEL,IIN,BIN_DESC
1,5859.0,France,United Kingdom,Visa,Credit,Consumer,NON-SECURE,497886.0,CLASSIC
2,5859.0,Bosnia and Herzegovina,United Kingdom,Mastercard,Debit,Consumer,NON-SECURE,535218.0,CLASSIC
3,5859.0,Sweden,United Kingdom,Mastercard,Credit,Consumer,NON-SECURE,522661.0,CLASSIC
4,5859.0,France,United Kingdom,Visa,Debit,Consumer,NON-SECURE,497040.0,CLASSIC
5,9391.0,United Kingdom,United Kingdom,Visa,Debit,Consumer,SECURE,465858.0,CLASSIC


In [15]:
bb1_contracts=bb1_contracts.drop_duplicates()

In [16]:
bb1_contracts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30742 entries, 1 to 358708
Data columns (total 9 columns):
CONTRACT_ID           30742 non-null float64
ISSUER_COUNTRY        30742 non-null object
MERCHANT_COUNTRY      30742 non-null object
CARD_SCHEME           30742 non-null object
CREDIT_DEBIT          30742 non-null object
CONSUMER_CORPORATE    30742 non-null object
SECURITY_LEVEL        30742 non-null object
IIN                   30742 non-null float64
BIN_DESC              30742 non-null object
dtypes: float64(2), object(7)
memory usage: 2.3+ MB


In [17]:
bb1_contracts.to_csv('d:\\djohnson\\Desktop\\bb1_contracts.csv')

## AIB Import

In [53]:
#name the columns
#all monetary is in euros
ab_column_names=['TRANSACTION_DATE','CONTRACT_ID','ISSUER_COUNTRY','CARD_SCHEME','MERCHANT_COUNTRY','TRANSACTION_TYPE','AIB_SF_RATE_AMOUNT','AIB_SF_PERC','AIB_SF_TOTAL','AIB_IC_RATE','AIB_IC_PERC','AIB_IC_TOTAL','AIB_SF_BPS','AIB_IC_BPS','CREDIT_DEBIT','CONSUMER_CORPORATE']

In [54]:
#import the file from local drives
ab=pd.read_csv('d:\\djohnson\\Desktop\\aib_comp_pull_aib_data_two.csv',error_bad_lines=False,skiprows=33,names=ab_column_names,sep=';',index_col=False)

In [55]:
ab.head(50)

,TRANSACTION_DATE,CONTRACT_ID,ISSUER_COUNTRY,CARD_SCHEME,MERCHANT_COUNTRY,TRANSACTION_TYPE,AIB_SF_RATE_AMOUNT,AIB_SF_PERC,AIB_SF_TOTAL,AIB_IC_RATE,AIB_IC_PERC,AIB_IC_TOTAL,AIB_SF_BPS,AIB_IC_BPS,CREDIT_DEBIT,CONSUMER_CORPORATE
0,09-NOV-17,1946,Australia,MasterCard,Luxembourg,Sale,4,0.113210,0.066170,0.0,1.60,0.113210,.09 ...,0.016543,Debit,Consumer ...
1,09-NOV-17,1946,Canada,MasterCard,Luxembourg,Sale,2,0.103145,0.033922,0.0,1.60,0.103145,.09 ...,0.016961,Credit,Consumer ...
2,14-NOV-17,2095,Russian Federation,Visa,Luxembourg,Sale,2,0.082426,0.035948,0.0,1.60,0.082426,.07 ...,0.017974,Debit,Consumer ...
3,26-DEC-17,1946,Latvia,MasterCard,Luxembourg,Sale,2,0.040924,0.004740,0.0,0.20,0.040924,.03 ...,0.002370,Debit,Consumer ...
4,14-NOV-17,2095,Russian Federation,MasterCard,Luxembourg,Sale,3,0.044565,0.050495,0.0,1.64,0.044565,.03 ...,0.016832,Debit,Consumer ...
5,14-NOV-17,1946,Bangladesh,Visa,Luxembourg,Sale,4,0.091004,0.060174,0.0,1.60,0.091004,.07 ...,0.015043,Credit,Consumer ...
6,14-NOV-17,2095,Russian Federation,Visa,Luxembourg,Sale,5,0.097803,0.092152,0.0,1.80,0.097803,.07 ...,0.018430,Debit,Consumer ...
7,14-NOV-17,2095,Russian Federation,Visa,Luxembourg,Sale,6,0.102804,0.096732,0.0,1.60,0.102804,.07 ...,0.016122,Debit,Consumer ...
8,15-NOV-17,1946,Canada,Visa,Luxembourg,Sale,3,0.086397,0.047104,0.0,1.60,0.086397,.07 ...,0.015701,Credit,Consumer ...
9,14-NOV-17,2464,Israel,MasterCard,United Kingdom,Sale,144,0.757060,3.244758,0.0,2.25,0.757060,.03 ...,0.022533,NaN,Commercial ...


In [41]:
ab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573067 entries, 0 to 2573066
Data columns (total 22 columns):
TRANSACTION_DATE      object
CONTRACT_ID           int64
ISSUER_COUNTRY        object
IIN                   object
BIN_DESC              object
CARD_SCHEME           object
MERCHANT_COUNTRY      int64
TRANSACTION_TYPE      float64
SECURITY_LEVEL        float64
CARD_TYPE             float64
IC_DESCRIPTION        float64
FLOW                  float64
AIB_SF_RATE_AMOUNT    object
AIB_SF_PERC           float64
AIB_SF_TOTAL          object
AIB_IC_RATE           object
AIB_IC_PERC           float64
AIB_IC_TOTAL          float64
AIB_SF_BPS            float64
AIB_IC_BPS            float64
CREDIT_DEBIT          float64
CONSUMER_CORPORATE    float64
dtypes: float64(12), int64(2), object(8)
memory usage: 431.9+ MB


In [42]:
ab['ISSUER_COUNTRY']=ab['ISSUER_COUNTRY'].str.strip()
ab['IIN']=ab['IIN'].str.strip()
ab['CARD_SCHEME']=ab['CARD_SCHEME'].str.strip()
ab['MERCHANT_COUNTRY']=ab['MERCHANT_COUNTRY'].str.strip()
ab['TRANSACTION_TYPE']=ab['TRANSACTION_TYPE'].str.strip()
ab['SECURITY_LEVEL']=ab['SECURITY_LEVEL'].str.strip()
ab['CARD_TYPE']=ab['CARD_TYPE'].str.strip()
ab['IC_DESCRIPTION']=ab['IC_DESCRIPTION'].str.strip()
ab['CREDIT_DEBIT']=ab['CREDIT_DEBIT'].str.strip()
ab['CONSUMER_CORPORATE']=ab['CONSUMER_CORPORATE'].str.strip()
ab['SECURITY_LEVEL']=ab['SECURITY_LEVEL'].str.strip()
ab['CONTRACT_ID']=ab['CONTRACT_ID'].astype(float)
ab['BIN_DESC']=ab['BIN_DESC'].str.strip()

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [ ]:
#create a new copy and safe old in memory for working purposes and to prevent repeated import from local drive
ab1=ab.copy()

In [21]:
ab1['SECURITY_LEVEL'].unique()

array(['Channel encrypted', 'Not relevant'], dtype=object)

In [22]:
ab1['SECURITY_LEVEL']=ab['SECURITY_LEVEL'].apply(lambda x:'NON-SECURE' if ('Not authenticated security transaction at a merchant who supports Verified-by-VISA - 3D-Secure' == x) or ('Not authenticated security transaction at a merchant who supports MasterCard SecureCode - UCAF' == x) or ('Channel encrypted' == x) or ('Not relevant' == x) else 'SECURE')

In [23]:
ab1['CREDIT_DEBIT']=ab['CREDIT_DEBIT'].apply(lambda x:'Debit' if ('Prepaid' == x) or ('Debit'== x) else 'Credit')

In [24]:
ab1.head()

,TRANSACTION_DATE,CONTRACT_ID,ISSUER_COUNTRY,IIN,BIN_DESC,CARD_SCHEME,MERCHANT_COUNTRY,TRANSACTION_TYPE,SECURITY_LEVEL,CARD_TYPE,...,AIB_SF_RATE_AMOUNT,AIB_SF_PERC,AIB_SF_TOTAL,AIB_IC_RATE,AIB_IC_PERC,AIB_IC_TOTAL,AIB_SF_BPS,AIB_IC_BPS,CREDIT_DEBIT,CONSUMER_CORPORATE
0,09-NOV-17,2095.0,Kazakhstan,405255,CLASSIC,Visa,Luxembourg,Sale,NON-SECURE,Visa Classic Debit,...,.07,.54,0.083736,0.0,1.6,0.041887,0.027912,0.013962,Debit,Consumer
1,14-NOV-17,1946.0,Spain,494019,CLASSIC,Visa,Luxembourg,Sale,NON-SECURE,Visa Classic Debit,...,.01,.02,0.021032,0.0,0.2,0.020640,0.002103,0.002064,Debit,Consumer
2,09-NOV-17,1946.0,United States,443041,CLASSIC,Visa,Luxembourg,Sale,NON-SECURE,Visa Classic Debit,...,.07,.54,0.084484,0.0,1.6,0.042554,0.028161,0.014185,Debit,Consumer
3,09-NOV-17,1946.0,Latvia,516897,CLASSIC,MasterCard,Luxembourg,Sale,NON-SECURE,Mastercard Debit Standard Card,...,.03,.059,0.055605,0.0,0.2,0.079123,0.001390,0.001978,Debit,Consumer
4,09-NOV-17,1946.0,Poland,460309,CLASSIC,Visa,Luxembourg,Sale,NON-SECURE,Visa Gold,...,.01,.03,0.020517,0.0,0.3,0.008725,0.006839,0.002908,Credit,Consumer


In [25]:
ab2=ab1[['CONTRACT_ID','ISSUER_COUNTRY','MERCHANT_COUNTRY','CARD_SCHEME','CREDIT_DEBIT','CONSUMER_CORPORATE','SECURITY_LEVEL','BIN_DESC']].drop_duplicates()

In [26]:
ab2.to_csv('d:\\djohnson\\Desktop\\ab2.csv')

In [27]:
ab1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4273736 entries, 0 to 4273735
Data columns (total 22 columns):
TRANSACTION_DATE      object
CONTRACT_ID           float64
ISSUER_COUNTRY        object
IIN                   int64
BIN_DESC              object
CARD_SCHEME           object
MERCHANT_COUNTRY      object
TRANSACTION_TYPE      object
SECURITY_LEVEL        object
CARD_TYPE             object
IC_DESCRIPTION        object
FLOW                  int64
AIB_SF_RATE_AMOUNT    object
AIB_SF_PERC           object
AIB_SF_TOTAL          float64
AIB_IC_RATE           float64
AIB_IC_PERC           float64
AIB_IC_TOTAL          float64
AIB_SF_BPS            float64
AIB_IC_BPS            float64
CREDIT_DEBIT          object
CONSUMER_CORPORATE    object
dtypes: float64(7), int64(2), object(13)
memory usage: 717.3+ MB


In [55]:
ab1=ab1[['CONTRACT_ID','ISSUER_COUNTRY','MERCHANT_COUNTRY','BIN_DESC','SECURITY_LEVEL','CARD_SCHEME','CREDIT_DEBIT','CONSUMER_CORPORATE','FLOW','AIB_SF_RATE_AMOUNT','AIB_SF_PERC','AIB_SF_BPS','AIB_SF_TOTAL']].drop_duplicates()

In [56]:
ab1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2783626 entries, 0 to 4273733
Data columns (total 13 columns):
CONTRACT_ID           float64
ISSUER_COUNTRY        object
MERCHANT_COUNTRY      object
BIN_DESC              object
SECURITY_LEVEL        object
CARD_SCHEME           object
CREDIT_DEBIT          object
CONSUMER_CORPORATE    object
FLOW                  int64
AIB_SF_RATE_AMOUNT    object
AIB_SF_PERC           object
AIB_SF_BPS            float64
AIB_SF_TOTAL          float64
dtypes: float64(3), int64(1), object(9)
memory usage: 297.3+ MB


## Attach AIB Lanes to Active Bambora Lanes 
This is a left join that will show all active Bambora lanes on only those AIB lanes where there is congruence.

In [72]:
#ab_bb_combo=pd.merge(bb1_contracts,ab1,on=['CONTRACT_ID','ISSUER_COUNTRY','MERCHANT_COUNTRY','CARD_SCHEME','CREDIT_DEBIT','CONSUMER_CORPORATE','SECURITY_LEVEL','IIN'],how='left')
ab_bb_combo=pd.merge(bb1_contracts,ab1,on=['CONTRACT_ID','ISSUER_COUNTRY','MERCHANT_COUNTRY','CARD_SCHEME','CREDIT_DEBIT','CONSUMER_CORPORATE','SECURITY_LEVEL','BIN_DESC'],how='left')

In [73]:
ab_bb_combo.head()

,CONTRACT_ID,ISSUER_COUNTRY,MERCHANT_COUNTRY,CARD_SCHEME,CREDIT_DEBIT,CONSUMER_CORPORATE,SECURITY_LEVEL,IIN,BIN_DESC,FLOW,AIB_SF_RATE_AMOUNT,AIB_SF_PERC,AIB_SF_BPS,AIB_SF_TOTAL
0,5859.0,France,United Kingdom,Visa,Credit,Consumer,NON-SECURE,497886.0,CLASSIC,NaN,NaN,NaN,NaN,NaN
1,5859.0,Bosnia and Herzegovina,United Kingdom,Mastercard,Debit,Consumer,NON-SECURE,535218.0,CLASSIC,NaN,NaN,NaN,NaN,NaN
2,5859.0,Sweden,United Kingdom,Mastercard,Credit,Consumer,NON-SECURE,522661.0,CLASSIC,NaN,NaN,NaN,NaN,NaN
3,5859.0,France,United Kingdom,Visa,Debit,Consumer,NON-SECURE,497040.0,CLASSIC,NaN,NaN,NaN,NaN,NaN
4,9391.0,United Kingdom,United Kingdom,Visa,Debit,Consumer,SECURE,465858.0,CLASSIC,NaN,NaN,NaN,NaN,NaN


### Only show those lanes where there is activity for AIB and Bambora
This is done but filtering to only those results that have scheme fee information

In [74]:
ab_bb_combo=ab_bb_combo[(ab_bb_combo['AIB_SF_PERC'].notnull())]

In [76]:
ab_bb_combo.head()

,CONTRACT_ID,ISSUER_COUNTRY,MERCHANT_COUNTRY,CARD_SCHEME,CREDIT_DEBIT,CONSUMER_CORPORATE,SECURITY_LEVEL,IIN,BIN_DESC,FLOW,AIB_SF_RATE_AMOUNT,AIB_SF_PERC,AIB_SF_BPS,AIB_SF_TOTAL
17,6891.0,Poland,Germany,Visa,Debit,Consumer,NON-SECURE,469878.0,CLASSIC,9.0,.00852,.0002,0.002316,0.020845
18,6891.0,Poland,Germany,Visa,Debit,Consumer,NON-SECURE,469878.0,CLASSIC,9.0,,,0.002318,0.020865
19,6891.0,Poland,Germany,Visa,Debit,Consumer,NON-SECURE,469878.0,CLASSIC,9.0,.01,.0002,0.002318,0.020865
20,6891.0,Poland,Germany,Visa,Debit,Consumer,NON-SECURE,469878.0,CLASSIC,26.0,.01,.0002,0.000867,0.022550
21,6891.0,Poland,Germany,Visa,Debit,Consumer,NON-SECURE,469878.0,CLASSIC,9.0,.00852,.0002,0.002322,0.020895


### Only Show the Median aggregated by lane, contract ID, Credit/Debit and Card type, Flow

In [77]:
#ab_bb_combo=ab_bb_combo.groupby(['CONTRACT_ID','ISSUER_COUNTRY','CARD_SCHEME','MERCHANT_COUNTRY','CREDIT_DEBIT','CONSUMER_CORPORATE','FLOW','SECURITY_LEVEL','IIN'],as_index=False)[['AIB_IC_BPS','AIB_SF_BPS']].median()
ab_bb_combo=ab_bb_combo.groupby(['CONTRACT_ID','ISSUER_COUNTRY','CARD_SCHEME','MERCHANT_COUNTRY','CREDIT_DEBIT','CONSUMER_CORPORATE','FLOW','SECURITY_LEVEL','BIN_DESC'],as_index=False)[['AIB_SF_BPS']].median()

In [78]:
print ab_bb_combo.head()

   CONTRACT_ID ISSUER_COUNTRY CARD_SCHEME MERCHANT_COUNTRY CREDIT_DEBIT  \
0       2095.0     Kazakhstan        Visa       Luxembourg       Credit   
1       2095.0     Kazakhstan        Visa       Luxembourg       Credit   
2       2095.0     Kazakhstan        Visa       Luxembourg       Credit   
3       2095.0     Kazakhstan        Visa       Luxembourg       Credit   
4       2095.0     Kazakhstan        Visa       Luxembourg       Credit   

  CONSUMER_CORPORATE  FLOW SECURITY_LEVEL BIN_DESC  AIB_SF_BPS  
0           Consumer   0.0     NON-SECURE  CLASSIC    0.000000  
1           Consumer   1.0     NON-SECURE  CLASSIC    0.080877  
2           Consumer   2.0     NON-SECURE  CLASSIC    0.040941  
3           Consumer   3.0     NON-SECURE  CLASSIC    0.028591  
4           Consumer   4.0     NON-SECURE  CLASSIC    0.022879  


In [79]:
ab_bb_combo.to_csv('d:\\djohnson\\Desktop\\ab_bb_comp.csv')

In [80]:
ab_bb_combo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 978 entries, 0 to 977
Data columns (total 10 columns):
CONTRACT_ID           978 non-null float64
ISSUER_COUNTRY        978 non-null object
CARD_SCHEME           978 non-null object
MERCHANT_COUNTRY      978 non-null object
CREDIT_DEBIT          978 non-null object
CONSUMER_CORPORATE    978 non-null object
FLOW                  978 non-null float64
SECURITY_LEVEL        978 non-null object
BIN_DESC              978 non-null object
AIB_SF_BPS            978 non-null float64
dtypes: float64(3), object(7)
memory usage: 84.0+ KB


## Setup the Bambora Baseline Lanes, Flows, and SFs

In [81]:
base=ab_bb_combo.drop(['FLOW','AIB_SF_BPS'],axis=1).drop_duplicates()

In [82]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 0 to 976
Data columns (total 8 columns):
CONTRACT_ID           95 non-null float64
ISSUER_COUNTRY        95 non-null object
CARD_SCHEME           95 non-null object
MERCHANT_COUNTRY      95 non-null object
CREDIT_DEBIT          95 non-null object
CONSUMER_CORPORATE    95 non-null object
SECURITY_LEVEL        95 non-null object
BIN_DESC              95 non-null object
dtypes: float64(1), object(7)
memory usage: 6.7+ KB


In [83]:
bbbase=pd.merge(base,bb1,on=['CONTRACT_ID','ISSUER_COUNTRY','MERCHANT_COUNTRY','CARD_SCHEME','CREDIT_DEBIT','CONSUMER_CORPORATE','SECURITY_LEVEL'],how='left')

In [84]:
bbbase=bbbase.rename(columns={'ROUNDED_TRXN_AMOUNT':'FLOW','FLOW':'FLOW_2'})

In [85]:
bbbase.head()

,CONTRACT_ID,ISSUER_COUNTRY,CARD_SCHEME,MERCHANT_COUNTRY,CREDIT_DEBIT,CONSUMER_CORPORATE,SECURITY_LEVEL,BIN_DESC_x,TRANSACTION_REF,ADDITIONAL_REFERENCE_2,...,PRODUCT_CODE,SF_FIXED_FEE,SF_PERC_FEE,VOLUME,FLOW,SCHEME_TOTAL_FEE_EUR,FLOW_2,BB_IC_TOTAL,BB_IC_BPS,BB_SF_BPS
0,2095.0,Kazakhstan,Visa,Luxembourg,Credit,Consumer,NON-SECURE,CLASSIC,225662136209,2.256621e+11,...,100444.0,0.075992,0.005657,1.0,1.0,0.067593,1.164676,0.018511,0.015894,0.058036
1,2095.0,Russian Federation,Visa,Luxembourg,Credit,Consumer,NON-SECURE,CLASSIC,225543153535,2.255432e+11,...,100444.0,0.074130,0.005633,1.0,5.0,0.061389,4.521325,0.031256,0.006913,0.013578
2,2095.0,Russian Federation,Visa,Luxembourg,Credit,Consumer,NON-SECURE,CLASSIC,225543153522,2.255432e+11,...,100444.0,0.074130,0.005631,1.0,10.0,0.073982,9.777313,0.067592,0.006913,0.007567
3,2095.0,Russian Federation,Visa,Luxembourg,Credit,Consumer,NON-SECURE,CLASSIC,225615152975,2.256152e+11,...,100444.0,0.074907,0.005622,1.0,20.0,0.174194,20.307209,0.327318,0.016118,0.008578
4,2095.0,Russian Federation,Visa,Luxembourg,Credit,Consumer,NON-SECURE,CLASSIC,225615153007,2.256152e+11,...,100444.0,0.074907,0.005622,1.0,197.0,1.158986,197.115463,3.177170,0.016118,0.005880


In [86]:
bbbase=bbbase.drop('BIN_DESC_y',axis=1)

In [87]:
bbbase=bbbase.rename(columns={'BIN_DESC_x':'BIN_DESC'})

In [88]:
bbbase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37619 entries, 0 to 37618
Data columns (total 32 columns):
CONTRACT_ID               37619 non-null float64
ISSUER_COUNTRY            37619 non-null object
CARD_SCHEME               37619 non-null object
MERCHANT_COUNTRY          37619 non-null object
CREDIT_DEBIT              37619 non-null object
CONSUMER_CORPORATE        37619 non-null object
SECURITY_LEVEL            37619 non-null object
BIN_DESC                  37619 non-null object
TRANSACTION_REF           37619 non-null object
ADDITIONAL_REFERENCE_2    37619 non-null float64
PAYMENTREFERENCE          37619 non-null float64
BB_TRANSACTION_TYPE       37619 non-null object
MCC                       37619 non-null float64
IIN                       37619 non-null float64
ORDERID                   37619 non-null float64
PAYMENT_PROCESSOR         37619 non-null float64
MONTH                     37619 non-null float64
DAY                       37619 non-null float64
YEAR              

### Link AIB and Bambora

In [89]:
#ab_bb_combo=pd.merge(bbbase,ab_bb_combo,on=['CONTRACT_ID','ISSUER_COUNTRY','CARD_SCHEME','MERCHANT_COUNTRY','CREDIT_DEBIT','CONSUMER_CORPORATE','FLOW','SECURITY_LEVEL','IIN'],how='left')
ab_bb_combo=pd.merge(bbbase,ab_bb_combo,on=['CONTRACT_ID','ISSUER_COUNTRY','CARD_SCHEME','MERCHANT_COUNTRY','CREDIT_DEBIT','CONSUMER_CORPORATE','FLOW','SECURITY_LEVEL','BIN_DESC'],how='left')

In [90]:
ab_bb_combo.head()

,CONTRACT_ID,ISSUER_COUNTRY,CARD_SCHEME,MERCHANT_COUNTRY,CREDIT_DEBIT,CONSUMER_CORPORATE,SECURITY_LEVEL,BIN_DESC,TRANSACTION_REF,ADDITIONAL_REFERENCE_2,...,SF_FIXED_FEE,SF_PERC_FEE,VOLUME,FLOW,SCHEME_TOTAL_FEE_EUR,FLOW_2,BB_IC_TOTAL,BB_IC_BPS,BB_SF_BPS,AIB_SF_BPS
0,2095.0,Kazakhstan,Visa,Luxembourg,Credit,Consumer,NON-SECURE,CLASSIC,225662136209,2.256621e+11,...,0.075992,0.005657,1.0,1.0,0.067593,1.164676,0.018511,0.015894,0.058036,0.080877
1,2095.0,Russian Federation,Visa,Luxembourg,Credit,Consumer,NON-SECURE,CLASSIC,225543153535,2.255432e+11,...,0.074130,0.005633,1.0,5.0,0.061389,4.521325,0.031256,0.006913,0.013578,0.019473
2,2095.0,Russian Federation,Visa,Luxembourg,Credit,Consumer,NON-SECURE,CLASSIC,225543153522,2.255432e+11,...,0.074130,0.005631,1.0,10.0,0.073982,9.777313,0.067592,0.006913,0.007567,0.012403
3,2095.0,Russian Federation,Visa,Luxembourg,Credit,Consumer,NON-SECURE,CLASSIC,225615152975,2.256152e+11,...,0.074907,0.005622,1.0,20.0,0.174194,20.307209,0.327318,0.016118,0.008578,0.008912
4,2095.0,Russian Federation,Visa,Luxembourg,Credit,Consumer,NON-SECURE,CLASSIC,225615153007,2.256152e+11,...,0.074907,0.005622,1.0,197.0,1.158986,197.115463,3.177170,0.016118,0.005880,NaN


## Insert the Signal logic

In [91]:
ab_bb_combo['SF_DIFF_%']=(ab_bb_combo['BB_SF_BPS']-ab_bb_combo['AIB_SF_BPS'])/ab_bb_combo['AIB_SF_BPS']

In [92]:
ab_bb_combo['SF_SIGNAL']=''
ab_bb_combo['SF_SIGNAL']=ab_bb_combo['SF_DIFF_%'].apply(lambda x:'Signal' if (x>=.05) else '')

### Final Output

In [93]:
ab_bb_combo=ab_bb_combo[['PERIOD'
                         ,'TRANSACTION_REF'
                         ,'TRANSACTION_TYPE'
                         ,'CONTRACT_ID'
                         ,'BIN_DESC'
                         ,'ISSUER_COUNTRY'
                         ,'MERCHANT_COUNTRY'
                         ,'REGIONALITY'
                         ,'CARD_SCHEME'
                         ,'CONSUMER_CORPORATE'
                         ,'CREDIT_DEBIT'
                         ,'SECURITY_LEVEL'
                         ,'VOLUME'
                         ,'FLOW'
                         ,'SF_FIXED_FEE'
                         ,'SF_PERC_FEE'
                         ,'SCHEME_TOTAL_FEE_EUR'
                         ,'BB_IC_TOTAL'
                         ,'BB_SF_BPS'
                         ,'BB_IC_BPS'
                         ,'AIB_SF_BPS'
                         ,'SF_DIFF_%'
                         ,'SF_SIGNAL']]

In [94]:
ab_bb_combo.head()

,PERIOD,TRANSACTION_REF,TRANSACTION_TYPE,CONTRACT_ID,BIN_DESC,ISSUER_COUNTRY,MERCHANT_COUNTRY,REGIONALITY,CARD_SCHEME,CONSUMER_CORPORATE,...,FLOW,SF_FIXED_FEE,SF_PERC_FEE,SCHEME_TOTAL_FEE_EUR,BB_IC_TOTAL,BB_SF_BPS,BB_IC_BPS,AIB_SF_BPS,SF_DIFF_%,SF_SIGNAL
0,2018-01,225662136209,TRANSACTION,2095.0,CLASSIC,Kazakhstan,Luxembourg,Interregional,Visa,Consumer,...,1.0,0.075992,0.005657,0.067593,0.018511,0.058036,0.015894,0.080877,-0.282416,
1,2018-01,225543153535,TRANSACTION,2095.0,CLASSIC,Russian Federation,Luxembourg,Interregional,Visa,Consumer,...,5.0,0.074130,0.005633,0.061389,0.031256,0.013578,0.006913,0.019473,-0.302763,
2,2018-01,225543153522,TRANSACTION,2095.0,CLASSIC,Russian Federation,Luxembourg,Interregional,Visa,Consumer,...,10.0,0.074130,0.005631,0.073982,0.067592,0.007567,0.006913,0.012403,-0.389929,
3,2018-01,225615152975,TRANSACTION,2095.0,CLASSIC,Russian Federation,Luxembourg,Interregional,Visa,Consumer,...,20.0,0.074907,0.005622,0.174194,0.327318,0.008578,0.016118,0.008912,-0.037494,
4,2018-01,225615153007,TRANSACTION,2095.0,CLASSIC,Russian Federation,Luxembourg,Interregional,Visa,Consumer,...,197.0,0.074907,0.005622,1.158986,3.177170,0.005880,0.016118,NaN,NaN,


In [95]:
print 'total transactions'
print ab_bb_combo['TRANSACTION_REF'].count()
print 'total signals'
print ab_bb_combo['TRANSACTION_REF'][(ab_bb_combo['SF_SIGNAL']=='Signal')].count()

total transactions
37619
total signals
206


In [96]:
ab_bb_combo.to_csv('d:\\djohnson\\Desktop\\aib to bambora comparison sf.csv')